In [1]:
import geopandas as gpd
from srai.loaders.osm_loaders import OSMOnlineLoader

In [2]:
gdf = gpd.read_file("data/shp/REJSTAT_20231231.shp")
gdf = gdf.to_crs(epsg=4326)
geometry = gdf.geometry.union_all()

loader = OSMOnlineLoader()

In [3]:
osm_non_residential_building_types = [
    "commercial",
    "industrial",
    "kiosk",
    "office",
    "retail",
    "supermarket",
    "warehouse",
    "church",
    "chapel",
    "hospital",
    "school",
    "university",
    "public",
    "barn",
    "farm_auxiliary",
    "greenhouse",
    "stable",
    "boathouse",
    "bunker",
]

osm_residential_building_types = [
    "apartments",
    "bungalow",
    "cabin",
    "detached",
    "dormitory",
    "house",
    "residential",
    "semidetached_house",
    "hut",
    "yes",
]

In [ ]:
allotments = loader.load(geometry, {"landuse": "allotments"}).to_crs(epsg=4326)
allotments = allotments.geometry.union_all()

region_to_load = geometry.difference(allotments)

In [ ]:
non_residential_buildings = loader.load(
    region_to_load,
    {
        "building": osm_non_residential_building_types,
        "building:levels": True,
    },
).to_crs(epsg=2180)

In [ ]:
residential_buildings = loader.load(
    region_to_load,
    {
        "building": osm_residential_building_types,
        "building:levels": True,
    },
).to_crs(epsg=2180)

In [7]:
non_residential_buildings = non_residential_buildings[
    ~non_residential_buildings["building"].isna()
]
residential_buildings = residential_buildings[~residential_buildings["building"].isna()]

residential_buildings = residential_buildings[residential_buildings.area >= 40]

residential_buildings.loc[
    residential_buildings["building"] == "yes", "building"
] = "unspecified"

In [ ]:
residential_buildings.explore()

In [ ]:
import folium

stops_df = gpd.read_file("data/mapa.geojson")

m = folium.Map(location=[51.1078565, 17.04563742], zoom_start=15)

for _, row in stops_df.iterrows():
    name = row["name"]
    lat = row["geometry"].y
    lon = row["geometry"].x
    folium.Marker(location=[lat, lon], popup=name).add_to(m)
m

### Odleglosc budynków od przystanków

In [10]:
def find_buildings_distance(
    gdf_buildings: gpd.GeoDataFrame, gdf_stops: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    if gdf_buildings.crs != gdf_stops.crs:
        gdf_stops = gdf_stops.to_crs(gdf_buildings.crs)
    buildings = gdf_buildings.copy()

    def count_min_distance(geom: gpd.GeoSeries) -> float:
        dinstace = [geom.distance(point) for point in gdf_stops.geometry]
        return min(dinstace)

    buildings["distance_to_stop"] = buildings.geometry.apply(count_min_distance)

    return buildings

In [11]:
gdf_buildings_processed = find_buildings_distance(residential_buildings, stops_df)

In [39]:
gdf_buildings_processed = gdf_buildings_processed.drop("building:levels", axis=1)
gdf_buildings_processed = gdf_buildings_processed.to_crs(epsg=4326)
gdf_stops_processed = stops_df[["id", "geometry"]]

In [48]:
import ipywidgets as widgets
from branca.colormap import LinearColormap
from IPython.display import clear_output, display


def create_folium_map(gdf_buildings: gpd.GeoDataFrame, gdf_stops: gpd.GeoDataFrame) -> None:
    buildings = gdf_buildings.copy()

    base_map = folium.Map(location=[51.1078565, 17.04563742], zoom_start=15)

    colormap = LinearColormap(
        colors=["green", "yellow", "red"],
        vmin=buildings["distance_to_stop"].min(),
        vmax=buildings["distance_to_stop"].max(),
    )
    base_map.add_child(colormap)

    map_container = widgets.Output()

    def update_map(max_distance):
        with map_container:
            clear_output(wait=True)
            m = folium.Map(location=[51.1078565, 17.04563742], zoom_start=15)
            m.add_child(colormap)

            style_function = lambda x: {
                "fillColor": colormap(x["properties"]["distance_to_stop"]),
                "color": "black",
                "weight": 1,
                "fillOpacity": 0.0
                if x["properties"]["distance_to_stop"] <= max_distance
                else 1.0,
            }

            folium.GeoJson(
                buildings.__geo_interface__, style_function=style_function, name="Budynki"
            ).add_to(m)

            for idx, row in gdf_stops.iterrows():
                folium.CircleMarker(
                    location=[row.geometry.y, row.geometry.x],
                    radius=8,
                    color="red",
                    fill=True,
                    popup=f"Przystanek {idx}",
                ).add_to(m)

            folium.LayerControl().add_to(m)

            total_buildings = len(buildings)
            buildings_in_range = len(
                buildings[buildings["distance_to_stop"] <= max_distance]
            )

            info = f"""
            <div style="padding: 10px; background-color: white; opacity: 0.8; border-radius: 5px;">
            <h4>Statystyki:</h4>
            <p>Budynki w zasięgu: {buildings_in_range} z {total_buildings}</p>
            <p>Procent w zasięgu: {(buildings_in_range/total_buildings*100):.1f}%</p>
            <p>Maksymalna odległość: {max_distance}m</p>
            <p>Mapa z budynkami przekraczajacymi maksymalny zasięg:</p>
            </div>
            """

            legend = folium.Element(info)
            m.get_root().html.add_child(legend)

            display(m)

    update_map(500)

    slider = widgets.FloatSlider(
        value=500,
        min=0,
        max=2000,
        step=50,
        description="Maksymalna odległość (m):",
        continuous_update=False,
        style={"description_width": "initial"},
        layout={"width": "500px"},
    )
    widgets.interactive_output(update_map, {"max_distance": slider})

    display(map_container, slider)

In [ ]:
create_folium_map(gdf_buildings_processed, gdf_stops_processed)